# Embbeding Models
In this notebook we are expermenting on two embedding models. One is the default text-embedding-ada-002 and the other is text-embedding-3-large. By improving the embedding model we can improve the relevancy of the retrived documents.

In [1]:
# Importing libraries
import sys
from dotenv import load_dotenv
import pandas as pd
import os
import shutil
import json
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_experimental.text_splitter import SemanticChunker
from typing import List
import pandas as pd
sys.path.insert(1, '/home/jabez/week_11/Contract-Advisor-RAG')
load_dotenv()
sys.path.insert(1, '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/scripts')
import file_loader 
import pipelines 
import evaluation

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load JSON from file
json_path = '../filepath.json'

with open(json_path, 'r') as json_file:
    file_paths = json.load(json_file)
data_file_path = file_paths['data_file_path']
synthetic_test_data_path = file_paths['synthetic_test_data_path']

# loading data
data = file_loader.load_csv(data_file_path)

# loading synthetic test data
synthetic_test_data = pd.read_csv(synthetic_test_data_path)

# loading persist directory for smaller chunck vector db
persist_directory_for_smaller_chunck_vector_db = file_paths['persist_directory_for_smaller_chunck_vector_db']

# loading persist directory for larger chunck vector db
persist_directory_for_larger_chunk_vector_db = file_paths['persist_directory_for_larger_chunk_vector_db']

# loading persist directory for semantic vector db
persist_directory_for_semantic_vector_db = file_paths['persist_directory_for_semantic_vector_db']

# loading persist directory for deafult vector db
persist_directory_for_default_vector_db = file_paths['persist_directory_for_default_vector_db']

### text-embedding-ada-002

In [3]:
# Create or load a Chroma database
embeddings = OpenAIEmbeddings()
db_default = Chroma(persist_directory=persist_directory_for_default_vector_db, embedding_function=embeddings)

In [4]:
# Setting retriever
retriver = db_default.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Adding answer to test data from simple pipeline
file_path_with_answer = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data_with_answer.csv'
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(synthetic_test_data, pipelines.simple_pipeline, db_default, retriver)

# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Evaluating: 100%|██████████| 80/80 [00:56<00:00,  1.41it/s]


context_precision: 86.73%, faithfulness: 78.12%, answer_relevancy: 90.59%, context_recall: 74.58%


Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectionError(ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))")
Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectionError(ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))")
Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to POST https://api.smith.langchain.com/runs/batch  in LangSmith API. Please confirm your internet connection.. ConnectionError(ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))")
Failed to batch ingest runs: LangSmithConnectionError("Connection error caused failure to POST

### text-embedding-3-large

In [5]:
# Create or load a Chroma database
embeddings = OpenAIEmbeddings()
db_large = Chroma(persist_directory=persist_directory_for_smaller_chunck_vector_db, embedding_function=embeddings)

In [8]:
# Generate syntetic test data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv'
syntetic_test_data =evaluation.generate_syntetic_testdata(data, file_path)

# Loading syntetic test data
syntetic_test_data = pd.read_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv')

# Setting retriever
retriver = db_large.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Adding answer to test data from simple pipeline
file_path_with_answer = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data_with_answer.csv'
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(syntetic_test_data, pipelines.simple_pipeline, vectorstore_character, retriver)

# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

Filename and doc_id are the same for all nodes.                     
Evaluating: 100%|██████████| 80/80 [00:36<00:00,  2.18it/s]


context_precision: 88.17%, faithfulness: 75.8%, answer_relevancy: 84.84%, context_recall: 80.42%
